In [1]:
# Tips for for new testers of the tomohara aliases (see https://github.com/tomasohara/shell-scripts repo).
#
# Notes:
# - In general, a distinct test is a Jupyter cell with output.
#      $ echo "hola, mundo"
#      hola, mundo
# - If you need to do a series of commands for a particular test, use the '# Setup' comment
#      # Setup
#      echo "fu" > /tmp/fu.list
# - There is also a global setup, such as for aliases used trhoughout
#      # Global setup
#      alias num-users="who --short | cut -f1 -d' ' | uniq | wc -l"
# - The following Bash aliases are used
#     display-macros: show the alias or function definition (helpful for designing clever tests)
#     tracs-vars: display values of Bash variables (useful for debugging the tests)
# - The following Bash idiom is used and the end of tests:
#      [ condition ]; echo $?
#   It outputs 0 if and only if the test succeeds. For example,
#      [ $count -gt 0]; echo $?
#   will output 0 if the count is above 0.
# - For more details on this testing paradigm, see <repo>/tests/README.ipynb.
# - See <repo>/template.bash for a cheatsheet for common constructs used in the aliases.
# - Warning: Bash is a real bear! Be careful not to spend too much learning its cryptic syntax, etc.

In [2]:
# Global setup
# Note: Put any helper aliases for your tests here
alias current-time-in-seconds='date +%s'
alias num-sleep-processes='ps | grep sleep | wc -l'

In [3]:
#................................................................................

In [4]:
# Test sleep-for function
#
# note: The sleep-for function shows a message and sleeps for a given amount of time.
# This test uses the date command to capture elapsed time in seconds to verify sleep-for
# works as expected. The main complication comes in the Bash syntax for arithmetic.
#
## DEBUG
display-macros sleep-for

sleep-for () 
{ 
    local sec="$1";
    local msg="${2:-"delay for ${sec}s"}";
    echo "$msg";
    sleep "$sec"
}



In [5]:
# Setup
# Make sure 'sleep-for N' sleeeps for at least N seconds
#
# Sleep for specified seconds 
time_before=$(current-time-in-seconds)
delay=2
sleep-for $delay
#
# Get time difference
time_after=$(current-time-in-seconds)
time_diff=$(($time_after - $time_before))
## DEBUG:
trace-vars time_before time_after time_diff 1>&2

delay for 2s
time_before=1696576947; time_after=1696576949; time_diff=2; 


In [6]:
# Make sure at least required delay
[ $time_diff -ge $delay ]; echo $?

0


In [7]:
#................................................................................
# Test kill-sleep alias
#
# Note: this alias is simple, but testing it is complicated by the need to put a process 
# in the background and to ensure that nothing is already sleeping. In addition, the
# steps are split across cells to show what is going on. This is why '# Setup' is used.
#
## DEBUG
display-macros kill-sleep

alias kill-sleep='kill_em.sh sleep'



In [8]:
# Setup: make sure no sleep processes
num-sleep-processes

0


In [9]:
# Setup: start a sleeping process
sleep 30 &

[1] 87927


In [10]:
# Setup: make sure 1 sleep process
num-sleep-processes

1


In [11]:
# Kill the sleep process and make sure none running
kill-sleep >| /tmp/kill-sleep-$$.log
num_sleeping=$(num-sleep-processes)
[ $num_sleeping -eq 0 ]; echo $?

[1]+  Killed                  sleep 30
0
